In [1]:
import folium
import numpy as np
import pandas as pd
from folium.plugins import HeatMap
from folium import plugins
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.colors as mcolors

In [2]:
df_total=pd.read_csv("data/fullWithLocations_FINAL_large.csv",index_col=False)
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503976 entries, 0 to 503975
Data columns (total 44 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   503976 non-null  int64  
 1   LAT          503976 non-null  float64
 2   LON          503976 non-null  float64
 3   YEAR         503976 non-null  int64  
 4   DOY          503976 non-null  int64  
 5   EVLAND       503976 non-null  float64
 6   GWETPROF     503976 non-null  float64
 7   GWETROOT     503976 non-null  float64
 8   GWETTOP      503976 non-null  float64
 9   PRECSNO      503976 non-null  float64
 10  PRECTOTCORR  503976 non-null  float64
 11  QV2M         503976 non-null  float64
 12  RH2M         503976 non-null  float64
 13  RHOA         503976 non-null  float64
 14  T10M         503976 non-null  float64
 15  T10M_MAX     503976 non-null  float64
 16  T10M_MIN     503976 non-null  float64
 17  T2M          503976 non-null  float64
 18  T2M_MAX      503976 non-

## Ashraf Attributes are 
### PRECSNO
### PRECSNO
### PRECSNO small dataset


In [3]:
df=df_total[["LAT","LON","DOY","YEAR","PRECSNO","PRECTOTCORR"]]
df.head(30)

,LAT,LON,DOY,YEAR,PRECSNO,PRECTOTCORR
0,22.5,25.625,1,2019,0.0,0.00
1,22.5,26.250,1,2019,0.0,0.00
2,22.5,26.875,1,2019,0.0,0.00
3,22.5,27.500,1,2019,0.0,0.00
4,22.5,28.125,1,2019,0.0,0.00
5,22.5,28.750,1,2019,0.0,0.00
6,22.5,29.375,1,2019,0.0,0.00
7,22.5,30.000,1,2019,0.0,0.00
8,22.5,30.625,1,2019,0.0,0.00
9,22.5,31.250,1,2019,0.0,0.00


In [4]:

Q1 = df['PRECSNO'].quantile(0.25)
Q2 = df['PRECSNO'].quantile(0.5)
Q3 = df['PRECSNO'].quantile(0.75)
Q4 = df['PRECSNO'].quantile(0.9)
Q5 = df['PRECSNO'].quantile(0.999)
Mean=df["PRECSNO"].mean()

# Step 2: Calculate the IQR
IQR = Q3 - Q1

# Step 3: Define lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Step 4: Count outliers
outliers = df[(df['PRECSNO'] < lower_bound) | (df['PRECSNO'] > upper_bound)]
outlier_count = outliers.shape[0]

# Display outlier count
print(outlier_count)
print(Q1)
print(Q2)
print(Q3)
print(Q4)
print(Q5)
print(Mean)

108
0.0
0.0
0.0
0.0
0.0
2.515992825055161e-05


In [5]:


# Create the WOY column
df['WOY'] = ((df['DOY'] - 1) // 7) + 1  # Convert DOY to Week of Year (assuming DOY starts from 1)

# Group by LAT, LON, YEAR, WOY and aggregate
weekly_df = df.groupby(['LAT', 'LON', 'YEAR', 'WOY']).agg(
    AVG_PRECSNO=('PRECSNO', 'mean'),
    MAX_PRECSNO=('PRECSNO', 'max'),
    MIN_PRECSNO=('PRECSNO', 'min')
).reset_index()

# Display the new DataFrame
weekly_df=weekly_df.sort_values(by=["YEAR",'WOY',"LAT",'LON'],ascending=[True,True,True,True])
display(len(weekly_df))
weekly_df.to_csv("Weekly_PRECSNO.csv",index=False)


C:\Users\ahmed\AppData\Local\Temp\ipykernel_13308\3948369916.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['WOY'] = ((df['DOY'] - 1) // 7) + 1  # Convert DOY to Week of Year (assuming DOY starts from 1)


73140

In [6]:

# Load Data
data = pd.read_csv("Weekly_PRECSNO.csv")
data['WOY'] = pd.to_numeric(data['WOY'], errors='coerce')
print(data.dtypes)


LAT            float64
LON            float64
YEAR             int64
WOY              int64
AVG_PRECSNO    float64
MAX_PRECSNO    float64
MIN_PRECSNO    float64
dtype: object


In [ ]:

# Function to generate and display the heatmap
def display_filtered_map(year, week, save=False):
    clear_output(wait=True)  # Clear previous map while keeping widgets visible
    display(year_selector, week_selector, button, save_button)  # Keep input cells visible

    # Filter data based on YEAR and WOY, and remove points with AVG_PRECSNO = 0
    filtered_data = data[(data['YEAR'] == year) & (data['WOY'] == week) & (data['AVG_PRECSNO'] > 0)]

    if filtered_data.empty:
        print(f"No data available for Year {year}, Week {week}")
        return

    # Create Folium map centered around Egypt
    egypt_map = folium.Map(location=[26.8206, 30.8025], zoom_start=6)

    # Normalize AVG_PRECSNO for better heatmap visualization
    min_value = filtered_data['AVG_PRECSNO'].min()
    max_value = filtered_data['AVG_PRECSNO'].max()
    
    filtered_data['Intensity'] = filtered_data['AVG_PRECSNO'].apply(
        lambda x: (x - min_value) / (max_value - min_value) if max_value > min_value else 0.5
    )

    # Prepare heatmap data with normalized intensity
    heatmap_data = filtered_data[['LAT', 'LON', 'Intensity']].values.tolist()
    
    # Add heatmap layer
    HeatMap(heatmap_data, radius=10, blur=15, max_zoom=10).add_to(egypt_map)

    # Get top 5 highest precipitation points
    top_5_points = filtered_data.nlargest(5, 'AVG_PRECSNO')

    # Define a colormap from red (high values) to blue (low values)
    cmap = mcolors.LinearSegmentedColormap.from_list("custom", ["blue", "cyan", "yellow", "orange", "red"])
    
    # Function to map intensity to color
    def intensity_to_color(intensity):
        return mcolors.to_hex(cmap(intensity))

    # Add all points with colors matching heatmap
    for _, row in filtered_data.iterrows():
        color = intensity_to_color(row['Intensity'])
        is_top5 = any(row.equals(top_5) for _, top_5 in top_5_points.iterrows()) 

        folium.CircleMarker(
            location=[row['LAT'], row['LON']],
            radius=10 if is_top5 else 5,  # Larger radius for top 5
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6 if is_top5 else 0.4,  # Make top 5 bolder
            tooltip=f"PRECSNO: {row['AVG_PRECSNO']}"
        ).add_to(egypt_map)

    # Save map only when requested
    if save:
        filename = f"PRECSNO_heatmap_{year}_W{week}.html"
        egypt_map.save(filename)
        print(f"Heatmap saved as {filename}")
    
    # Display the map
    display(egypt_map)

# Input widgets
year_selector = widgets.IntText(
    value=data['YEAR'].min(),
    description="Year:",
    min=data['YEAR'].min(),
    max=data['YEAR'].max()
)

week_selector = widgets.IntText(
    value=1,
    description="Week:",
    min=1,
    max=52
)

button = widgets.Button(description="Show Heatmap")
save_button = widgets.Button(description="Save as HTML")

# Function to update map when button is clicked
def on_button_click(b):
    display_filtered_map(year_selector.value, week_selector.value)

button.on_click(on_button_click)

# Function to save map when save button is clicked
def on_save_button_click(b):
    display_filtered_map(year_selector.value, week_selector.value, save=True)

save_button.on_click(on_save_button_click)

# Display widgets
display(year_selector, week_selector, button, save_button)

IntText(value=2019, description='Year:')

IntText(value=1, description='Week:')

Button(description='Show Heatmap', style=ButtonStyle())

Button(description='Save as HTML', style=ButtonStyle())

In [ ]:

# Function to process each week and extract precipitation points
def extract_weekly_precipitation(data, year):
    results = []
    
    for week in range(1, data[data['YEAR'] == year]['WOY'].max() + 1):
        weekly_data = data[(data['YEAR'] == year) & (data['WOY'] == week) & (data['AVG_PRECSNO'] > 0)]
        
        if weekly_data.empty:
            print(f"Warning: No data for YEAR {year}, WOY {week}")
            results.append([week, year] + [None] * 15 + [None])
            continue
        
        top_5 = weekly_data.nlargest(min(5, len(weekly_data)), 'AVG_PRECSNO')[['LAT', 'LON', 'AVG_PRECSNO']]
        top_5['AVG_PRECSNO'] = top_5['AVG_PRECSNO'].round(3)
        avg_precip = round(weekly_data['AVG_PRECSNO'].mean(), 3)
        
        top_5_values = top_5.values.flatten().tolist()
        
        # Ensure list has 15 elements (5 points * 3 values each)
        while len(top_5_values) < 15:
            top_5_values.append(None)  # Fill missing values with None for clarity
        
        results.append([week, year] + top_5_values + [avg_precip])
    
    return results

# Process the data for all available years
all_results = []
years = data['YEAR'].unique()

for y in years:
    all_results.extend(extract_weekly_precipitation(data, y))

# Convert to DataFrame and save to CSV
columns = ['WOY', 'YEAR'] + [f'Top{i}_{attr}' for i in range(1, 6) for attr in ['LAT', 'LON', 'Value']] + ['Avg']
weekly_summary = pd.DataFrame(all_results, columns=columns)

# Replace None with empty strings to avoid NaNs in CSV
weekly_summary.fillna('', inplace=True)

weekly_summary.to_csv('weekly_precipitation_summary.csv', index=False)

print("CSV file 'weekly_precipitation_summary.csv' has been saved.")


CSV file 'weekly_precipitation_summary.csv' has been saved.


C:\Users\ahmed\AppData\Local\Temp\ipykernel_24760\43857262.py:39: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  weekly_summary.fillna('', inplace=True)
